In [1]:
import pymongo
import pandas as pd
from pymongo import MongoClient


import numpy as np
import pandas as pd
import scipy.stats as stats

import datetime

import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')

In [2]:
client = MongoClient()
db_aud = client.aud_scraper

collection_aud = db_aud.rates

In [3]:
df_aud = pd.DataFrame(list(collection_aud.find()))

In [4]:
df_aud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   _id            344 non-null    object
 1   date           341 non-null    object
 2   change%        343 non-null    object
 3   AUD_rate_tgt%  340 non-null    object
dtypes: object(4)
memory usage: 10.9+ KB


In [5]:
# remove Mongo _id column

df_aud.drop(columns = ['_id'], inplace = True)

In [6]:
# drop rows with NaN values

df_aud.dropna(inplace = True)

In [7]:
df_aud.head()

,date,change%,AUD_rate_tgt%
0,6 May 2020,0.00,0.25
1,8 Apr 2020,0.00,0.25
2,20 Mar 2020,-0.25,0.25
3,4 Mar 2020,-0.25,0.50
4,5 Feb 2020,0.00,0.75


In [8]:
df_aud.rename(columns = {'change%': 'aud_rate_dif'}, inplace = True)

In [9]:
def date_converter(date):
    
    """
    PARAMETERS
    date - string in format '2-digit-day month_name 4-digit-year'
    
    RETURNS
    date in new string format ' 4 digit year-2 digit month-2 digit day'
    """
    
    dct_month = {
        'Jan': '01',
        'Feb': '02',
        'Mar': '03',
        'Apr': '04',
        'May': '05',
        'Jun': '06',
        'Jul': '07',
        'Aug': '08',
        'Sep': '09',
        'Oct': '10',
        'Nov': '11',
        'Dec': '12'
    }
    
    lst_date = date.split()
        
    str_yr = lst_date[-1]
    
    str_mo = dct_month[lst_date[1]]
    
    str_d = lst_date[0]
    
    #add zeros before single digit dates
    if len(str_d) ==1:
        str_d += '0'
        str_d = str_d[::-1]
        
    
    str_date = f'{str_yr}-{str_mo}-{str_d}'

    return str_date

In [10]:
df_aud['date'] = [date_converter(d) for d in df_aud['date']]

In [11]:
df_aud['date'] = pd.to_datetime(df_aud['date'], yearfirst = True)

In [12]:
df_aud.sort_values(by = 'date', inplace = True)

In [13]:
df_aud.head()

,date,aud_rate_dif,AUD_rate_tgt%
339,1990-01-23,-0.50 to -1.00,17.00 to 17.50
338,1990-02-15,-0.50,16.50 to 17.00
337,1990-03-07,0.00,16.50 to 17.00
336,1990-04-04,-1.00 to -1.50,15.00 to 15.50
335,1990-05-02,0.00,15.00 to 15.50


In [14]:
df_aud['AUD_rate_tgt%'] = [r.split()[0] for r in df_aud['AUD_rate_tgt%']]
df_aud['aud_rate_dif'] = [r.split()[-1] for r in df_aud['aud_rate_dif']]


In [15]:
df_aud.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340 entries, 339 to 0
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           340 non-null    datetime64[ns]
 1   aud_rate_dif   340 non-null    object        
 2   AUD_rate_tgt%  340 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 10.6+ KB


In [16]:
df_aud['AUD_rate_tgt%'] = pd.to_numeric(df_aud['AUD_rate_tgt%'])
df_aud['aud_rate_dif'] = pd.to_numeric(df_aud['aud_rate_dif'])

In [17]:
df_aud['AUD_rate_tgt%_lag1'] = df_aud['AUD_rate_tgt%'].shift(1)

In [18]:
df_aud['aud_rate_change%'] = 100*((df_aud['AUD_rate_tgt%'] - df_aud['AUD_rate_tgt%_lag1']) / df_aud['AUD_rate_tgt%_lag1'])

In [19]:
df_aud.head()

,date,aud_rate_dif,AUD_rate_tgt%,AUD_rate_tgt%_lag1,aud_rate_change%
339,1990-01-23,-1.0,17.0,NaN,NaN
338,1990-02-15,-0.5,16.5,17.0,-2.941176
337,1990-03-07,0.0,16.5,16.5,0.000000
336,1990-04-04,-1.5,15.0,16.5,-9.090909
335,1990-05-02,0.0,15.0,15.0,0.000000


In [20]:
%store df_aud

Stored 'df_aud' (DataFrame)
